In [1]:
import numpy as np
import pandas as pd 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Import classification dataset
train = pd.read_csv('train.csv')

In [3]:
# remove the p_id column since this column is not needed 
X=train.drop(['p_id', 'diabetes'], axis=1)

#Get the target columns (diabetes)
y=train.diabetes.values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [5]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.81      0.88      0.85       121
           1       0.74      0.61      0.67        64

    accuracy                           0.79       185
   macro avg       0.77      0.75      0.76       185
weighted avg       0.78      0.79      0.78       185



### Experiment 2: Train Random Forest Classifier

In [7]:

# Initiating the classifier
rf_clf = RandomForestClassifier(n_estimators=100)

# Passing the data to classifier
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.83      0.84      0.84       121
           1       0.69      0.67      0.68        64

    accuracy                           0.78       185
   macro avg       0.76      0.76      0.76       185
weighted avg       0.78      0.78      0.78       185



### Experiment 3: Train XGBoost

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81       121
           1       0.64      0.69      0.66        64

    accuracy                           0.76       185
   macro avg       0.73      0.74      0.74       185
weighted avg       0.76      0.76      0.76       185



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [9]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1], dtype=int64), array([261, 261], dtype=int64))

In [10]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80       121
           1       0.62      0.64      0.63        64

    accuracy                           0.74       185
   macro avg       0.71      0.72      0.72       185
weighted avg       0.74      0.74      0.74       185



### Track Experiments Using MLFlow

In [11]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [12]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [13]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [14]:
# Initialize MLflow
mlflow.set_experiment("Diabetes Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model") 

2024/09/07 13:33:21 INFO mlflow.tracking.fluent: Experiment with name 'Diabetes Detection' does not exist. Creating a new experiment.
2024/09/07 13:33:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/07 13:33:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/637006991232950144/runs/2a33c7e679e04aa0814d366e3cc5910a.
2024/09/07 13:33:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/637006991232950144.
2024/09/07 13:33:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/07 13:33:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://localhost:5000/#/experiments/6